<a href="https://colab.research.google.com/github/dagpadraget/rakdnngit/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /

/


In [2]:
cd content

/content


In [0]:
!rm -fr rakdnngit
!rm -fr tmp

In [4]:
!git clone https://github.com/dagpadraget/rakdnngit

Cloning into 'rakdnngit'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 98 (delta 47), reused 72 (delta 23), pack-reused 0
Unpacking objects: 100% (98/98), done.


In [5]:
cd /content

/content


In [0]:
mkdir tmp

In [7]:
cd tmp

/content/tmp


In [8]:
!mkdir rakdnngit
!ls -la
!cd /content/rakdnngit

total 12
drwxr-xr-x 3 root root 4096 Jan  5 19:27 .
drwxr-xr-x 1 root root 4096 Jan  5 19:27 ..
drwxr-xr-x 2 root root 4096 Jan  5 19:27 rakdnngit


In [9]:
cd /content/tmp

/content/tmp


In [10]:
!pwd

/content/tmp


In [11]:
!cd /content/tmp
!ls
!pwd

rakdnngit
/content/tmp


In [12]:
cd /content/rakdnngit

/content/rakdnngit


In [13]:
!ls -la

total 7356
drwxr-xr-x 5 root root    4096 Jan  5 19:27 .
drwxr-xr-x 1 root root    4096 Jan  5 19:27 ..
-rw-r--r-- 1 root root 7410460 Jan  5 19:27 dagmymodel.model
drwxr-xr-x 8 root root    4096 Jan  5 19:27 .git
drwxr-xr-x 2 root root    4096 Jan  5 19:27 localdata
-rw-r--r-- 1 root root   85940 Jan  5 19:27 main.ipynb
-rw-r--r-- 1 root root     513 Jan  5 19:27 main.py
-rw-r--r-- 1 root root     178 Jan  5 19:27 requirement.txt
-rw-r--r-- 1 root root     245 Jan  5 19:27 run_local
drwxr-xr-x 2 root root    4096 Jan  5 19:27 trainer


In [14]:
!pip3 install -r requirement.txt

In [15]:
!python3 main.py

Using TensorFlow backend.
number of sequences= 141 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-01-05 19:28:24.948948: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-01-05 19:28:24.962429: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-01-05 19:28:24.962954: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-01-05 19:28:24.963176: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-01-05 19:28:24.964617: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


In [16]:
pwd

'/content/rakdnngit'

In [0]:
more localdata/hbshort.txt

In [18]:
ls -rla /content/tmp/rakdnngit/jobdir/rak_dnngit

total 7248
-rw-r--r-- 1 root root 7410460 Jan  5 19:28 allincluded.model
drwxr-xr-x 3 root root    4096 Jan  5 19:28 ../
drwxr-xr-x 2 root root    4096 Jan  5 19:28 ./


In [19]:
# *********************************************************************************************************************
# Library imports
import numpy as np
import os
from keras.preprocessing.text import Tokenizer
from keras.layers import Input, Conv2D,LeakyReLU, Flatten, Dense,Reshape,Activation,Conv2DTranspose,Embedding,Dropout,LSTM
from keras.models import Model
from keras.models import load_model
from keras.optimizers import RMSprop
from keras.utils import np_utils
import re
import argparse
# *********************************************************************************************************************


Using TensorFlow backend.


In [0]:
ext_seq_length=4

In [0]:
# *********************************************************************************************************************
# Load data from file
def load_data(datapath):
    filename=datapath+"/hblang.txt"
    with open(filename,encoding="utf-8-sig") as f:
        text =f.read()

    seq_length=ext_seq_length # sequence around sentence
    step=1 # prediction step forward

    start_story='| '*seq_length

    text=text.lower()
    text=start_story+text
    text=text.replace('\n\n\n\n\n',start_story)
    text=text.replace('\n',' ')
    #text=re.sub(' +','. ',text).strip()
    text=text.replace('..','.')
    text=re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_{|}~])',r' \1 ',text)
    text=re.sub('\s{2,}',' ',text)

    tokenizer=Tokenizer(char_level=False,filters='')
    tokenizer.fit_on_texts([text])
    total_words=len(tokenizer.word_index)+1
    token_list=tokenizer.texts_to_sequences([text])[0]

    return token_list, seq_length,step,total_words,start_story,tokenizer
# *********************************************************************************************************************

In [0]:
def generate_text2(seed_text,next_words,model,max_sequence_len,temp,start_story,tokenizer):
    token_list = np.array(tokenizer.texts_to_sequences([seed_text])[0])
    token_list=token_list[-ext_seq_length:]
    probs = model.predict(token_list.reshape(1,len(token_list)), verbose=0)[0]
    
    print(1)
    p0=-np.sort(-probs)
    print(2)
    i0=np.argsort(-probs)
    print(3)
    print("words probablility",p0[0:9])
    print("words index=",i0[0:9])
    print(4)
    pos=0
    for k in i0:
      print("words in order=",k," : ",tokenizer.index_word[k],'p=',p0[pos])
      if pos>10:
        break
      pos+=1
    print(5)
    
    y_class = sample_with_temp(probs, temperature=temp)
    output_word = tokenizer.index_word[y_class] if y_class > 0 else ''
    return output_word


In [0]:
# *********************************************************************************************************************
def sample_with_temp(preds,temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probs=np.random.multinomial(1,preds,1)
    return np.argmax(probs)
# *********************************************************************************************************************


In [0]:
def test_model(args):
    fileandpath = os.path.join(args.job_dir, 'rak_dnngit')+"/allincluded.model"

    try:
        model=load_model(fileandpath)
        model.summary()

        token_list, seq_length, step, total_words,start_story,tokenizer = load_data("localdata")
        print(1)
        seedtext="till handelsman och"
        print(2)
        newtext=generate_text2(seedtext,1,model,10,0.01,start_story,tokenizer)
        print(3)
        print("output from loaded model : (",seedtext,") => ", newtext)
    except:
        print("could not find "+fileandpath)
  
    return model

In [0]:
def nextw(model,args,seedtext):
    fileandpath = os.path.join(args.job_dir, 'rak_dnngit')+"/allincluded.model"

    try:
        token_list, seq_length, step, total_words,start_story,tokenizer = load_data("localdata")
        newtext=generate_text2(seedtext,1,model,10,0.01,start_story,tokenizer)
        print("output from loaded model : (",seedtext,") => ", newtext)
    except:
        print("could not find "+fileandpath)
  
    return newtext

In [0]:
tmpdirname='/content/tmp/rakdnngit'

In [0]:
xbatch_size=32
xlearning_rate=0.01
xnum_epochs=1


In [0]:
class Argar:
    pcsim = True
    pcsimdir = tmpdirname
    batch_size=xbatch_size
    learning_rate=xlearning_rate
    num_epochs=xnum_epochs
    job_dir=tmpdirname+"/jobdir"

In [28]:
argen=Argar()
model=test_model(argen)
print("*************** test main - done")


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         8700      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         365568    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
___________________________________________

In [32]:
nextw(model,argen,"handelsman och ha")

1
2
3
words probablility [9.7405690e-01 1.1880924e-02 8.4086424e-03 4.2987545e-03 7.5667095e-04
 2.8790551e-04 1.3319797e-04 2.9075010e-05 1.8518569e-05]
words index= [45 56  3 53 73 85 12 21 59]
4
words in order= 45  :  kunde p= 0.9740569
words in order= 56  :  från p= 0.011880924
words in order= 3  :  . p= 0.008408642
words in order= 53  :  _ p= 0.0042987545
words in order= 73  :  : p= 0.00075667095
words in order= 85  :  andra p= 0.0002879055
words in order= 12  :  med p= 0.00013319797
words in order= 21  :  de p= 2.907501e-05
words in order= 59  :  ja p= 1.8518569e-05
words in order= 1  :  , p= 1.6006112e-05
words in order= 7  :  som p= 9.351805e-06
words in order= 70  :  här p= 8.5701495e-06
5
output from loaded model : ( handelsman och ha ) =>  kunde


'kunde'